#Settings

In [ ]:
%pip install pyopencl
%pip install pyparticles

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 917 kB 5.0 MB/s 
     |████████████████████████████████| 71 kB 9.5 MB/s 
  Created wheel for pytools: filename=pytools-2022.1.13-py2.py3-none-any.whl size=66024 sha256=8db68777ff86cd7c60462871115076a9e4fcb23484f433a5121805090cbd0205
  Stored in directory: /root/.cache/pip/wheels/b5/c1/bb/26ba70fb9d10f195249ef4e170a92ae83e7534e55b67786fd9
Successfully built pytools


In [ ]:
import pyopencl


In [ ]:
pyopencl.VERSION


(2022, 3)

In [ ]:
import pyopencl as cl
kcode = """ kernel void test () { printf (" Hi! (%d)\\n" , get_group_id (0)); } """
ctx = cl.create_some_context ()
Q = cl.CommandQueue (ctx)
prg = cl.Program (ctx, kcode).build (options ="")
prg.test (Q, [8], [1]) . wait ()

In [ ]:
from pyopencl.tools import get_test_platforms_and_devices
get_test_platforms_and_devices()

[(<pyopencl.Platform 'NVIDIA CUDA' at 0x38b9810>,
  [<pyopencl.Device 'Tesla T4' on 'NVIDIA CUDA' at 0x38b98b0>])]

In [ ]:
import os
os.environ ['PYOPENCL_COMPILER_OUTPUT'] = '1'
os.environ ['PYOPENCL_CTX'] = '0:0'

In [ ]:
import numba
print(numba.__version__)
from numba import jit
from numpy import arange
@jit
def Sum2D ( arr ):
    M , N = arr . shape
    result = 0.0
    for i in range ( M ):
        for j in range ( N ):
            result += arr [i , j]
    return result
a = arange ( 9 ) . reshape (3 , 3 )
print ( Sum2D ( a ) )

0.56.4
36.0


In [ ]:
pyopencl.get_cl_header_version()

(3, 0)

In [ ]:
pyopencl.get_platforms()

[<pyopencl.Platform 'NVIDIA CUDA' at 0x38b9810>]

In [ ]:
#python.exe -m pip install --upgrade pip #Для Анаконда промпт


#gravity_cluster

In [ ]:
import pyparticles.pset.rand_cluster as clu
import pyparticles.forces.gravity as gr

import pyparticles.ode.euler_solver as els

import pyparticles.pset.particles_set as ps
import pyparticles.pset.opencl_context as occ 

import pyparticles.animation.animated_ogl as aogl

from pyparticles.utils.pypart_global import test_pyopencl

import numpy as np

def gravity_cluster():
    
    if not test_pyopencl() :
        print("")
        print("Attention !!! ")
        print(" This demo works only with PyOpenCL: \n  http://mathema.tician.de/software/pyopencl \n  http://www.khronos.org/opencl/ \n ")
        print(" Please install the package: python-pyopencl for continuing")
        print("")
        return 
        
    
    G = 0.000001
    steps = 100000000
    
    n = 3000
    dt = 0.04
    
    pset = ps.ParticlesSet( n , dtype=np.float32 ) 
        
    cs = clu.RandGalaxyCluster()
    
    print("Building initial galaxy .... ")
    cs.insert3( pset.X, M=pset.M, V=pset.V , G=G )
    
    try :
        occx = occ.OpenCLcontext(  pset.size , pset.dim , (occ.OCLC_X|occ.OCLC_V|occ.OCLC_A|occ.OCLC_M) )
    except :
        print("")
        print("ERROR !!! ")
        print(" Please verify your opencl installation ")
        print(" Probably you must install your GPU OpenCL drivers")
        print("")
        return 
    
    grav = gr.GravityOCL( pset.size , Consts=G , ocl_context=occx  )
    grav.set_masses( pset.M )
    
    grav.update_force( pset )
    
    solver = els.EulerSolverOCL( grav , pset , dt , ocl_context=occx )
    
    a = aogl.AnimatedGl()
        
    a.draw_particles.set_draw_model( a.draw_particles.DRAW_MODEL_VECTOR )
        
    a.ode_solver = solver
    a.pset = pset
    a.steps = steps
    
    a.build_animation()
    
    a.start()
    
    return

In [ ]:
gravity_cluster()

ValueError: ignored

#pseudo_bubble

In [ ]:

import os
#os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

import pyparticles.pset.particles_set as ps

import pyparticles.forces.pseudo_bubble as pb
import pyparticles.forces.const_force as cf
import pyparticles.forces.drag as dr
import pyparticles.forces.multiple_force as mf

import pyparticles.ode.euler_solver as els
import pyparticles.ode.leapfrog_solver as lps
import pyparticles.ode.runge_kutta_solver as rks
import pyparticles.ode.stormer_verlet_solver as svs
import pyparticles.ode.midpoint_solver as mds

import pyparticles.pset.rand_cluster as rc
import pyparticles.pset.rebound_boundary as rb

import pyparticles.animation.animated_ogl as aogl

import numpy as np

from pyparticles.utils.pypart_global import test_pyopencl
import pyparticles.pset.opencl_context as occ 


def bubble():
    """
    Pseudo bubble simulation
    """
    
    ocl_ok = test_pyopencl()
    
    if ocl_ok :
        pcnt = 9000
        r_min=0.5
        occx = occ.OpenCLcontext( pcnt , 3  )
    else :
        pcnt = 700
        r_min = 1.5
    
    steps = 1000000
    dt = 0.01
    
    
    rand_c = rc.RandCluster()
    
    pset = ps.ParticlesSet( pcnt , dtype=np.float32 )
    
    rand_c.insert3( X=pset.X ,
                    M=pset.M ,
                    start_indx=0 ,
                    n=pset.size ,
                    radius=3.0 ,
                    mass_rng=(0.5,0.8) ,
                    r_min=0.0 )
    
    if ocl_ok :
        bubble = pb.PseudoBubbleOCL( pset.size , pset.dim , Consts=(r_min,10) )
        drag = dr.DragOCL( pset.size , pset.dim , Consts=0.01 )
    else :
        bubble = pb.PseudoBubble( pset.size , pset.dim , Consts=(r_min,10) )
        drag = dr.Drag( pset.size , pset.dim , Consts=0.01 )

    constf = cf.ConstForce( pset.size , dim=pset.dim , u_force=[ 0 , 0 , -10.0 ] )
    
    multif = mf.MultipleForce( pset.size , pset.dim )
    multif.append_force( bubble )
    multif.append_force( constf )
    multif.append_force( drag )
    
    multif.set_masses( pset.M )
    
    #solver = els.EulerSolver( multif , pset , dt )
    solver = lps.LeapfrogSolver( multif , pset , dt )
    #solver = svs.StormerVerletSolver( multif , pset , dt )
    #solver = rks.RungeKuttaSolver( lennard_jones , pset , dt )    
    #solver = mds.MidpointSolver( lennard_jones , pset , dt ) 
    
    bound = rb.ReboundBoundary( bound=(-5.0,5.0) )
    
    pset.set_boundary( bound )
    
    a = aogl.AnimatedGl()
    
    a.ode_solver = solver
    a.pset = pset
    a.steps = steps
    
    if ocl_ok :
        a.draw_particles.set_draw_model( a.draw_particles.DRAW_MODEL_VECTOR )
        
    a.init_rotation( -80 , [ 0.7 , 0.05 , 0 ]  )
    
    a.build_animation()
    
    a.start()
    
    return

In [ ]:
bubble()

RuntimeError: ignored

#Param platform

In [ ]:
import pyopencl as cl

In [ ]:
def print_device_info() :
    print('\n' + '=' * 60 + '\nOpenCL Platforms and Devices')
    for platform in cl.get_platforms():
        print('=' * 60)
        print('Platform - Name: ' + platform.name)
        print('Platform - Vendor: ' + platform.vendor)
        print('Platform - Version: ' + platform.version)
        print('Platform - Profile: ' + platform.profile)

        for device in platform.get_devices():
            print(' ' + '-' * 56)
            print(' Device - Name: ' \
                  + device.name)
            print(' Device - Type: ' \
                  + cl.device_type.to_string(device.type))
            print(' Device - Max Clock Speed: {0} Mhz'\
                  .format(device.max_clock_frequency))
            print(' Device - Compute Units: {0}'\
                  .format(device.max_compute_units))
            print(' Device - Local Memory: {0:.0f} KB'\
                  .format(device.local_mem_size/1024.0))
            print(' Device - Constant Memory: {0:.0f} KB'\
                  .format(device.max_constant_buffer_size/1024.0))
            print(' Device - Global Memory: {0:.0f} GB'\
                  .format(device.global_mem_size/1073741824.0))
            print(' Device - Max Buffer/Image Size: {0:.0f} MB'\
                  .format(device.max_mem_alloc_size/1048576.0))
            print(' Device - Max Work Group Size: {0:.0f}'\
                  .format(device.max_work_group_size))
    print('\n')

In [ ]:
if __name__ == "__main__":
    print_device_info()


OpenCL Platforms and Devices
Platform - Name: NVIDIA CUDA
Platform - Vendor: NVIDIA Corporation
Platform - Version: OpenCL 1.2 CUDA 11.2.109
Platform - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Tesla T4
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1590 Mhz
 Device - Compute Units: 40
 Device - Local Memory: 48 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 15 GB
 Device - Max Buffer/Image Size: 3777 MB
 Device - Max Work Group Size: 1024




#plus vector

In [ ]:
import numpy as np
import pyopencl as cl
import numpy.linalg as la

In [ ]:
vector_dimension = 100

In [ ]:
vector_a = np.random.randint(vector_dimension,size=vector_dimension) 
vector_b = np.random.randint(vector_dimension,size=vector_dimension)

In [ ]:
platform = cl.get_platforms()[0]
device = platform.get_devices()[0]
context = cl.Context([device])
queue = cl.CommandQueue(context)

In [ ]:
mf = cl.mem_flags 
a_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_a) 
b_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=vector_b) 

In [ ]:
program = cl.Program(context, """ 
__kernel void vectorSum(__global const int *a_g, __global const int *b_g, __global int *res_g) { 
  int gid = get_global_id(0); 
  res_g[gid] = a_g[gid] + b_g[gid]; 
} 
""").build()

In [ ]:
res_g = cl.Buffer(context, mf.WRITE_ONLY, vector_a.nbytes)

In [ ]:
program.vectorSum(queue, vector_a.shape, None, a_g, b_g, res_g)

In [ ]:
res_np = np.empty_like(vector_a)

In [ ]:
cl.enqueue_copy(queue, res_np, res_g)

In [ ]:
print ("PyOPENCL SUM OF TWO VECTORS")
print ("Platform Selected = %s" %platform.name )
print ("Device Selected = %s" %device.name)
print ("VECTOR LENGTH = %s" %vector_dimension)
print ("INPUT VECTOR A")
print (vector_a)
print ("INPUT VECTOR B")
print (vector_b)
print ("OUTPUT VECTOR RESULT A + B ")
print (res_np)

PyOPENCL SUM OF TWO VECTORS
Platform Selected = NVIDIA CUDA
Device Selected = Tesla T4
VECTOR LENGTH = 100
INPUT VECTOR A
[83 77 52  4 42 64 87 46 13 97 14  1 69  6 31 44 93  4  6 76 21 85 57 76
 82 67 38 30 12 80 70 45 59 31 96  9 28 21  0 42 65 69 96 10 27 17 80 50
 72 72 99 88 68 91 94 82 89 55 78 34  8 53 39 50 21 79 47 58 81  2 89 54
 80  0 60 62 36  4 82 56 67 59 75 72 92 52 13 94  0  6 54 33 91 22 19 88
 92 59 91 73]
INPUT VECTOR B
[74 60 45 12 99 58 87 84 41 69 65 19 53 20  7 30 30 71  6 79 18 94 64 11
 36 67 61 74 14 18 33 11 87 60 90 27 63 32 14 58 26 72 75  9 29 57 62 90
 75 90 19 25 87  7 65  4 62 79 38 24 78 55 71 72 76 95 37 81 22 55 75 65
 45 99 21 26  4  3 99 15 65 86 42 25 93 64  7 33 56 89 95 62 51 82 30 56
 21 46 43 24]
OUTPUT VECTOR RESULT A + B 
[157 137  97  16 141 122 174 130  54 166  79  20 122  26  38  74 123  75
  12 155  39 179 121  87 118 134  99 104  26  98 103  56 146  91 186  36
  91  53  14 100  91 141 171  19  56  74 142 140 147 162   0   0   0   0
   0

#plus vector gpu and cpu

In [ ]:
from time import time  # Import time tools

import pyopencl as cl  
import numpy as np   
#import deviceInfoPyopencl as device_info
import numpy.linalg as la

#input vectors 
a = np.random.rand(10000).astype(np.float32)  
b = np.random.rand(10000).astype(np.float32)   

def test_cpu_vector_sum(a, b):  
    c_cpu = np.empty_like(a)   
    cpu_start_time = time()  
    for i in range(10000):
            for j in range(10000):  
                    c_cpu[i] = a[i] + b[i]  
    cpu_end_time = time()   
    print("CPU Time: {0} s".format(cpu_end_time - cpu_start_time))   
    return c_cpu   

def test_gpu_vector_sum(a, b):
    #define the PyOpenCL Context
    platform = cl.get_platforms()[0]
    device = platform.get_devices()[0]
    context = cl.Context([device])
    queue = cl.CommandQueue(context, \
                            properties=cl.command_queue_properties.PROFILING_ENABLE)   
    #prepare the data structure
    a_buffer = cl.Buffer\
               (context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=a)
    b_buffer = cl.Buffer\
               (context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=b)
    c_buffer = cl.Buffer\
               (context, cl.mem_flags.WRITE_ONLY, b.nbytes)   
    program = cl.Program(context, """
    __kernel void sum(__global const float *a, __global const float *b, __global float *c)
    {
        int i = get_global_id(0);
        int j;
        for(j = 0; j < 10000; j++)
        {
            c[i] = a[i] + b[i];
        }
    }""").build()
    #start the gpu test
    gpu_start_time = time()   
    event = program.sum(queue, a.shape, None, a_buffer, b_buffer, c_buffer)   
    event.wait()   
    elapsed = 1e-9*(event.profile.end - event.profile.start)   
    print("GPU Kernel evaluation Time: {0} s".format(elapsed))   
    c_gpu = np.empty_like(a)  
    cl._enqueue_read_buffer(queue, c_buffer, c_gpu).wait()  
    gpu_end_time = time()  
    print("GPU Time: {0} s".format(gpu_end_time - gpu_start_time))   
    return c_gpu   

#start the test
if __name__ == "__main__":
    #print the device info
  #  device_info.print_device_info()
    #call the test on the cpu
    cpu_result = test_cpu_vector_sum(a, b)
    #call the test on the gpu
    gpu_result = test_gpu_vector_sum(a, b)
    assert (la.norm(cpu_result - gpu_result)) < 1e-5

CPU Time: 31.39669132232666 s
GPU Kernel evaluation Time: 0.0012363200000000002 s
GPU Time: 0.003478527069091797 s


In [ ]:
import pyopencl as cl
import pyopencl.array as cl_array
import numpy
import numpy.linalg as la

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

n = 10
a_gpu = cl_array.to_device(queue,
        (numpy.random.randn(n) + 1j*numpy.random.randn(n)).astype(numpy.complex64))
b_gpu = cl_array.to_device(queue,
        (numpy.random.randn(n) + 1j*numpy.random.randn(n)).astype(numpy.complex64))

from pyopencl.elementwise import ElementwiseKernel
complex_prod = ElementwiseKernel(ctx,
        "float a, "
        "cfloat_t *x, "
        "cfloat_t *y, "
        "cfloat_t *z",
        "z[i] = cfloat_rmul(a, cfloat_mul(x[i], y[i]))",
        "complex_prod",
        preamble="#include <pyopencl-complex.h>")

complex_add = ElementwiseKernel(ctx,
        "cfloat_t *x, "
        "cfloat_t *y, "
        "cfloat_t *z",
        "z[i] = cfloat_add(x[i], y[i])",
        "complex_add",
        preamble="#include <pyopencl-complex.h>")

real_part = ElementwiseKernel(ctx,
        "cfloat_t *x, float *z",
        "z[i] = cfloat_real(x[i])",
        "real_part",
        preamble="#include <pyopencl-complex.h>")

c_gpu = cl_array.empty_like(a_gpu)
complex_prod(5, a_gpu, b_gpu, c_gpu)

c_gpu_real = cl_array.empty(queue, len(a_gpu), dtype=numpy.float32)
real_part(c_gpu, c_gpu_real)
print(c_gpu.get().real - c_gpu_real.get())

print(la.norm(c_gpu.get() - (5*a_gpu.get()*b_gpu.get())))
assert la.norm(c_gpu.get() - (5*a_gpu.get()*b_gpu.get())) < 1e-5

RuntimeError: ignored